In [1]:
# Extraer imagenes para entrenar modelo de reconocimiento de dígitos.

In [2]:
# Cargar librerías.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from fake_useragent import UserAgent
import keys as k
import time
import cv2

import warnings
warnings.filterwarnings('ignore')


## Cargar funciones

In [3]:
# Página principal con la data
url = "https://operaciones.sisfoh.gob.pe:450/cse/"
# Abrir la página web
# Setear opciones de enmascaramiento
options = webdriver.ChromeOptions();
#Enmascarar fuente de salida de la consulta.
#headers = {"user-agent": k.user_agent}
userAgent = (UserAgent()).random
options.add_argument(f'user-agent={userAgent}')
#options.add_argument('user-agent':'{}'.format(k.user_agent))
options.add_argument('--incognito')
# Configurar el driver que abrirá la web
driver = webdriver.Chrome('chromedriver', chrome_options=options)
# Abrir la web
driver.get(url)
time.sleep(1)
# Cerrar mensaje emergente
driver.find_element_by_xpath("//*[@class = 'close']").click()
time.sleep(1)

In [4]:
# Encontrar captcha con 3 digitos
def get_images(driver,i):
    driver.save_screenshot('images2/image_{}.png'.format(str(i)))
    # Cargar imagen y quedarse solamente con el área de interés.
    image = cv2.imread('images2/image_{}.png'.format(str(i), 1))
    # Recortar área de interés
    image = image[485:540, 330:480]
    cv2.imwrite('images2/image_{}.png'.format(str(i)), image)


In [5]:
for i in range(1001,2001):
    try:
        get_images(driver,i)
        driver.find_element_by_xpath("//*[@class = 'refrescar']").click()
        time.sleep(2)
    except:
        print('error')
        driver.find_element_by_xpath("//*[@class = 'refrescar']").click()
        time.sleep(2)
driver.quit()

-----------

# Etiquetar imágenes

In [15]:
import cv2
import pytesseract as tes
import re

In [35]:
def get_captcha(index):
    # Cargar imagen.
    img = cv2.imread(f'images/image_{index}.png', 1)
    # Convertir imagen a texto
    tes.pytesseract.tesseract_cmd = r'C:\Users\alaboriano\AppData\Local\Programs\Tesseract-OCR\tesseract.exe'
    text = tes.image_to_string(img)
    # Descifrar captcha.
    #captcha = int(re.sub('[a-zA-Z]', '', re.sub('[^\w\s]','',text)).strip())
    captcha = re.sub('[a-zA-Z]', '', re.sub('[^\w\s]','',text).strip()).strip()
    return captcha
 

In [39]:
import os
anotaciones = []
for index in range(1,1001,1):
    #print(image)
    label = get_captcha(index)
    #print(tmp)
    anotaciones.append({'image':'image' + '_' + str(index),'label':label})

In [40]:
pd.DataFrame(anotaciones)

,image,label
0,image_1,7
1,image_2,852
2,image_3,213
3,image_4,685
4,image_5,
...,...,...
995,image_996,
996,image_997,
997,image_998,852
998,image_999,651


In [41]:
import pandas as pd
pd.DataFrame(anotaciones).to_csv('anotaciones_pytesseract_original.csv', index = False)

In [42]:
import pandas as pd
pd.DataFrame(anotaciones).to_csv('anotaciones.csv', index = False)